In [70]:
# Gerekli kütüphaneler
import pandas as pd
import numpy as np
from datetime import datetime

In [71]:
CSV_PATH = "match_odds_cleaned_20250801.csv"
KEY_COLS = [
    "match_date", "match_time", "tournament", "match_id",
    "homeTeam", "awayTeam",
    "firstHalfHomeGoal", "firstHalfAwayGoal",
    "totalHomeGoal", "totalAwayGoal",
    "homeCorner", "awayCorner"
]


In [72]:
df = pd.read_csv(f"data/processed/{CSV_PATH}")
df = df.sample(frac=1)
numeric_cols = [col for col in df.columns if col not in KEY_COLS]
train_set = df.dropna(subset=["totalHomeGoal", "totalAwayGoal"])
train_set.shape

(4220, 495)

In [73]:
test_set = df[df[["totalHomeGoal", "totalAwayGoal"]].isna().any(axis=1)]
test_set.shape

(489, 495)

In [74]:
test_row = test_set.iloc[0]
test_row

match_date                                                          2025-08-29
match_time                                                            02:00:00
tournament                                                   Şili Primera Ligi
match_id                                                               2261627
homeTeam                                                            Everton DV
                                                                   ...        
İlk Yarı Çifte Şans :: İY 1-X                                              1.0
İlk Yarı Çifte Şans :: İY X-2                                             1.27
İlk Yarıda Hangi Takım Daha Fazla Korner Kullanır :: Dep                   NaN
İlk Yarıda Hangi Takım Daha Fazla Korner Kullanır :: Ev                    NaN
İlk Yarıda Hangi Takım Daha Fazla Korner Kullanır :: Eşit                  NaN
Name: 4598, Length: 495, dtype: object

In [75]:
# 3. Özellik sütunlarını belirle
feature_cols = [col for col in df.columns if col not in KEY_COLS]

# Sayısal verileri al (kategorik varsa elleme hatası olur)
feature_cols = [col for col in feature_cols if pd.api.types.is_numeric_dtype(df[col])]


In [76]:
# Train: skoru olanlar, Test: skoru olmayanlar
train_df = df[df["totalHomeGoal"].notna()].copy()
test_df  = df[df["totalHomeGoal"].isna()].copy()

# Sadece oran sütunları
ODDS_COLS = [col for col in df.columns if col not in KEY_COLS]

train_odds = train_df[ODDS_COLS]
test_odds = test_df[ODDS_COLS]

In [77]:
# -- 2. Testten rastgele bir maç seç
import random

random_idx = random.randint(0, len(test_df)-1)
test_row = test_df.iloc[random_idx]

In [78]:
from collections import defaultdict

def group_bet_columns(df):
    odds_cols = [col for col in df.columns if "::" in col]
    bet_groups = defaultdict(list)

    for col in odds_cols:
        bet_type, option = col.split("::", maxsplit=1)
        bet_groups[bet_type.strip()].append(col)

    return dict(bet_groups)


In [79]:
value_bets =  pd.DataFrame(columns=["match_date", "match_time", "match_id", "hometeam", "awayteam", "bet_name", "probability", "odds"])

In [80]:
train_df = df[df["totalHomeGoal"].notna()].reset_index(drop=True)
test_df = df[df["totalHomeGoal"].isna()].reset_index(drop=True)
# -- 3. En yakın 100 maçı bul

flag = 0

for idx_test, test_row in test_df.iterrows():
    today = pd.to_datetime(datetime.today().date())
    match_date = pd.to_datetime(test_row["match_date"])

    bets = {}
    distances = []
    
    # 3. Geçmiş maçsa atla
    # print(match_date < today)
    # if test_row["match_id"] != 2284625:
        # continue
    
    # if match_date < today:
    #     continue
    
    for idx, train_row in train_df.iterrows():
        train_vals = train_row[numeric_cols]
        test_vals = test_row[numeric_cols]
        
        # test_vals = test_vals.iloc[0]

        valid_mask = ~(train_vals.isna() | test_vals.isna())

        if valid_mask.sum() == 0:
            dist = np.inf
        else:
            diff_squared = (train_vals[valid_mask] - test_vals[valid_mask]) ** 2
            dist = np.sqrt(diff_squared.sum())

        distances.append((idx, dist))

    distance_df = pd.DataFrame(distances, columns=["index", "distance"])
    # distance_df.to_csv(f"distances_{test_row['match_id']}.csv", index=False)
    top_100_indices = distance_df.sort_values(by="distance").head(100)

    # ✅ Fixed indexing issue here
    top_100_matches = train_df.loc[top_100_indices["index"]]
    # top_100_matches.to_csv(f"top_100_matches_{test_row["match_id"]}.csv", index=False)

    # Bütün bahis tipleri
    home_goal = top_100_matches["totalHomeGoal"]
    away_goal = top_100_matches["totalAwayGoal"]
    first_half_home_goal = top_100_matches["firstHalfHomeGoal"]
    first_half_away_goal = top_100_matches["firstHalfAwayGoal"]
    second_half_home_goal = home_goal - first_half_home_goal
    second_half_away_goal = away_goal - first_half_away_goal

    

    
    # -- 4. MS 1-X-2 sonuçlarını hesapla
    ms1_count = (home_goal > away_goal).sum()
    msx_count = ((top_100_matches["totalHomeGoal"] == top_100_matches["totalAwayGoal"])).sum()
    ms2_count = ((top_100_matches["totalHomeGoal"] < top_100_matches["totalAwayGoal"])).sum()
    print(ms1_count, msx_count, ms2_count)

    # Deplasman Gol Alt/Üst
    bets["Deplasman Gol Alt/Üst :: Dep 0,5 Alt"] = ((away_goal < 0.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 0,5 Alt"]
    bets["Deplasman Gol Alt/Üst :: Dep 0,5 Üst"] = ((away_goal >= 0.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 0,5 Üst"]
    bets["Deplasman Gol Alt/Üst :: Dep 1,5 Alt"] = ((away_goal < 1.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 1,5 Alt"]
    bets["Deplasman Gol Alt/Üst :: Dep 1,5 Üst"] = ((away_goal >= 1.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 1,5 Üst"]
    bets["Deplasman Gol Alt/Üst :: Dep 2,5 Alt"] = ((away_goal < 2.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 2,5 Alt"]
    bets["Deplasman Gol Alt/Üst :: Dep 2,5 Üst"] = ((away_goal >= 2.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 2,5 Üst"]
    bets["Deplasman Gol Alt/Üst :: Dep 3,5 Alt"] = ((away_goal < 3.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 3,5 Alt"]
    bets["Deplasman Gol Alt/Üst :: Dep 3,5 Üst"] = ((away_goal >= 3.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 3,5 Üst"]
    bets["Deplasman Gol Alt/Üst :: Dep 4,5 Alt"] = ((away_goal < 4.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 4,5 Alt"]
    bets["Deplasman Gol Alt/Üst :: Dep 4,5 Üst"] = ((away_goal >= 4.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 4,5 Üst"]
    bets["Deplasman Gol Alt/Üst :: Dep 6,5 Alt"] = ((away_goal < 6.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 6,5 Alt"]
    bets["Deplasman Gol Alt/Üst :: Dep 6,5 Üst"] = ((away_goal >= 6.5)).sum(), test_row["Deplasman Gol Alt/Üst :: Dep 6,5 Üst"]

    # Deplasman Gol Yemeden Kazanır mı?
    bets["Deplasman Gol Yemeden Kazanır mı? :: Evet"] = ((away_goal > home_goal) & (home_goal == 0)).sum(), test_row["Deplasman Gol Yemeden Kazanır mı? :: Evet"]
    bets["Deplasman Gol Yemeden Kazanır mı? :: Hayır"] = (~((away_goal > home_goal) & (home_goal == 0))).sum(), test_row["Deplasman Gol Yemeden Kazanır mı? :: Hayır"]

    # Deplasman Hangi Yarıda Daha Fazla Gol Atar?
    bets["Deplasman Hangi Yarıda Daha Fazla Gol Atar? :: 1. Yarı"] = ((first_half_away_goal > second_half_away_goal)).sum(), test_row["Deplasman Hangi Yarıda Daha Fazla Gol Atar? :: 1. Yarı"]
    bets["Deplasman Hangi Yarıda Daha Fazla Gol Atar? :: 2. Yarı"] = ((second_half_away_goal > first_half_away_goal)).sum(), test_row["Deplasman Hangi Yarıda Daha Fazla Gol Atar? :: 2. Yarı"]
    bets["Deplasman Hangi Yarıda Daha Fazla Gol Atar? :: Eşit"] = ((first_half_away_goal == second_half_away_goal)).sum(), test_row["Deplasman Hangi Yarıda Daha Fazla Gol Atar? :: Eşit"]

    # Deplasman Her İki Yarıyı da Kazanır mı?
    bets["Deplasman Her İki Yarıyı da Kazanır mı? :: Evet"] = (
        ((first_half_away_goal > first_half_home_goal) & (second_half_away_goal > second_half_home_goal)).sum(),
        test_row["Deplasman Her İki Yarıyı da Kazanır mı? :: Evet"]
    )

    bets["Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır"] = (
        (~((first_half_away_goal > first_half_home_goal) & (second_half_away_goal > second_half_home_goal))).sum(),
        test_row["Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır"]
    )

    # Deplasman Herhangi Bir Yarıyı Kazanır mı?
    bets["Deplasman Herhangi Bir Yarıyı Kazanır :: Evet"] = (
        ((first_half_away_goal > first_half_home_goal) | (second_half_away_goal > second_half_home_goal)).sum(),
        test_row["Deplasman Herhangi Bir Yarıyı Kazanır :: Evet"]
    )

    bets["Deplasman Herhangi Bir Yarıyı Kazanır :: Hayır"] = (
        (~((first_half_away_goal > first_half_home_goal) | (second_half_away_goal > second_half_home_goal))).sum(),
        test_row["Deplasman Herhangi Bir Yarıyı Kazanır :: Hayır"]
    )

    # Deplasman İlk Yarı Gol Alt/Üst
    threshold_map = {
        "0,5": 0.5,
        "1,5": 1.5,
        "2,5": 2.5
    }

    for label, value in threshold_map.items():
        bets[f"Deplasman İlk Yarı Gol Alt/Üst :: Dep İY {label} Alt"] = (
            (first_half_away_goal < value).sum(),
            test_row[f"Deplasman İlk Yarı Gol Alt/Üst :: Dep İY {label} Alt"]
        )
        bets[f"Deplasman İlk Yarı Gol Alt/Üst :: Dep İY {label} Üst"] = (
            (first_half_away_goal >= value).sum(),
            test_row[f"Deplasman İlk Yarı Gol Alt/Üst :: Dep İY {label} Üst"]
        )

    # Ev Sahibi Gol Alt/Üst
    threshold_map = {
        "0,5": 0.5,
        "1,5": 1.5,
        "2,5": 2.5,
        "3,5": 3.5,
        "4,5": 4.5
    }

    for label, value in threshold_map.items():
        bets[f"Ev Sahibi Gol Alt/Üst :: Ev {label} Alt"] = (
            (home_goal < value).sum(),
            test_row[f"Ev Sahibi Gol Alt/Üst :: Ev {label} Alt"]
        )
        bets[f"Ev Sahibi Gol Alt/Üst :: Ev {label} Üst"] = (
            (home_goal >= value).sum(),
            test_row[f"Ev Sahibi Gol Alt/Üst :: Ev {label} Üst"]
        )

    # Ev Sahibi Gol Yemeden Kazanır mı?
    bets["Ev Sahibi Gol Yemeden Kazanır mı? :: Evet"] = (
        ((home_goal > away_goal) & (away_goal == 0)).sum(),
        test_row["Ev Sahibi Gol Yemeden Kazanır mı? :: Evet"]
    )

    bets["Ev Sahibi Gol Yemeden Kazanır mı? :: Hayır"] = (
        (~((home_goal > away_goal) & (away_goal == 0))).sum(),
        test_row["Ev Sahibi Gol Yemeden Kazanır mı? :: Hayır"]
    )

    # Ev Sahibi Hangi Yarıda Daha Fazla Gol Atar?
    bets["Ev Sahibi Hangi Yarıda Daha Fazla Gol Atar? :: 1. Yarı"] = (
        (first_half_home_goal > second_half_home_goal).sum(),
        test_row["Ev Sahibi Hangi Yarıda Daha Fazla Gol Atar? :: 1. Yarı"]
    )

    bets["Ev Sahibi Hangi Yarıda Daha Fazla Gol Atar? :: 2. Yarı"] = (
        (second_half_home_goal > first_half_home_goal).sum(),
        test_row["Ev Sahibi Hangi Yarıda Daha Fazla Gol Atar? :: 2. Yarı"]
    )

    bets["Ev Sahibi Hangi Yarıda Daha Fazla Gol Atar? :: Eşit"] = (
        (first_half_home_goal == second_half_home_goal).sum(),
        test_row["Ev Sahibi Hangi Yarıda Daha Fazla Gol Atar? :: Eşit"]
    )

    # Ev Sahibi Her İki Yarıyı da Kazanır mı?
    bets["Ev Sahibi Her İki Yarıyı da Kazanır mı? :: Evet"] = (
        ((first_half_home_goal > first_half_away_goal) & (second_half_home_goal > second_half_away_goal)).sum(),
        test_row["Ev Sahibi Her İki Yarıyı da Kazanır mı? :: Evet"]
    )

    bets["Ev Sahibi Her İki Yarıyı da Kazanır mı? :: Hayır"] = (
        (~((first_half_home_goal > first_half_away_goal) & (second_half_home_goal > second_half_away_goal))).sum(),
        test_row["Ev Sahibi Her İki Yarıyı da Kazanır mı? :: Hayır"]
    )

    # Ev Sahibi Herhangi Bir Yarıyı Kazanır mı?
    bets["Ev Sahibi Herhangi Bir Yarıyı Kazanır :: Evet"] = (
        ((first_half_home_goal > first_half_away_goal) | (second_half_home_goal > second_half_away_goal)).sum(),
        test_row["Ev Sahibi Herhangi Bir Yarıyı Kazanır :: Evet"]
    )

    bets["Ev Sahibi Herhangi Bir Yarıyı Kazanır :: Hayır"] = (
        (~((first_half_home_goal > first_half_away_goal) | (second_half_home_goal > second_half_away_goal))).sum(),
        test_row["Ev Sahibi Herhangi Bir Yarıyı Kazanır :: Hayır"]
    )

    # Ev Sahibi İlk Yarı Gol Alt/Üst
    threshold_map = {
        "0,5": 0.5,
        "1,5": 1.5,
        "2,5": 2.5
    }

    for label, value in threshold_map.items():
        bets[f"Ev Sahibi İlk Yarı Gol Alt/Üst :: Ev İY {label} Alt"] = (
            (first_half_home_goal < value).sum(),
            test_row[f"Ev Sahibi İlk Yarı Gol Alt/Üst :: Ev İY {label} Alt"]
        )
        bets[f"Ev Sahibi İlk Yarı Gol Alt/Üst :: Ev İY {label} Üst"] = (
            (first_half_home_goal >= value).sum(),
            test_row[f"Ev Sahibi İlk Yarı Gol Alt/Üst :: Ev İY {label} Üst"]
        )

    # Hangi Takım Kaç Farkla Kazanır
    goal_diff = home_goal - away_goal

    bets["Hangi Takım Kaç Farkla Kazanır :: Berabere"] = (
        (goal_diff == 0).sum(),
        test_row["Hangi Takım Kaç Farkla Kazanır :: Berabere"]
    )

    bets["Hangi Takım Kaç Farkla Kazanır :: Dep 1 Fark"] = (
        (goal_diff == -1).sum(),
        test_row["Hangi Takım Kaç Farkla Kazanır :: Dep 1 Fark"]
    )

    bets["Hangi Takım Kaç Farkla Kazanır :: Dep 2 Fark"] = (
        (goal_diff == -2).sum(),
        test_row["Hangi Takım Kaç Farkla Kazanır :: Dep 2 Fark"]
    )

    bets["Hangi Takım Kaç Farkla Kazanır :: Dep 3+ Fark"] = (
        (goal_diff <= -3).sum(),
        test_row["Hangi Takım Kaç Farkla Kazanır :: Dep 3+ Fark"]
    )

    bets["Hangi Takım Kaç Farkla Kazanır :: Ev 1 Fark"] = (
        (goal_diff == 1).sum(),
        test_row["Hangi Takım Kaç Farkla Kazanır :: Ev 1 Fark"]
    )

    bets["Hangi Takım Kaç Farkla Kazanır :: Ev 2 Fark"] = (
        (goal_diff == 2).sum(),
        test_row["Hangi Takım Kaç Farkla Kazanır :: Ev 2 Fark"]
    )

    bets["Hangi Takım Kaç Farkla Kazanır :: Ev 3+ Fark"] = (
        (goal_diff >= 3).sum(),
        test_row["Hangi Takım Kaç Farkla Kazanır :: Ev 3+ Fark"]
    )

    # Hangi Yarıda Daha Fazla Gol Atılır?
    first_half_total_goal = first_half_home_goal + first_half_away_goal
    second_half_total_goal = second_half_home_goal + second_half_away_goal

    bets["Hangi Yarıda Daha Fazla Gol Atılır? :: 1. Yarı"] = (
        (first_half_total_goal > second_half_total_goal).sum(),
        test_row["Hangi Yarıda Daha Fazla Gol Atılır? :: 1. Yarı"]
    )

    bets["Hangi Yarıda Daha Fazla Gol Atılır? :: 2. Yarı"] = (
        (second_half_total_goal > first_half_total_goal).sum(),
        test_row["Hangi Yarıda Daha Fazla Gol Atılır? :: 2. Yarı"]
    )

    bets["Hangi Yarıda Daha Fazla Gol Atılır? :: Eşit"] = (
        (first_half_total_goal == second_half_total_goal).sum(),
        test_row["Hangi Yarıda Daha Fazla Gol Atılır? :: Eşit"]
    )

    # Her İki Yarıda da 1.5 Gol Alt/Üst Olur mu?
    first_half_total_goal = first_half_home_goal + first_half_away_goal
    second_half_total_goal = second_half_home_goal + second_half_away_goal

    # ALT
    bets["Her İki Yarıda da 1.5 Gol Alt Olur mu? :: Evet"] = (
        ((first_half_total_goal < 1.5) & (second_half_total_goal < 1.5)).sum(),
        test_row["Her İki Yarıda da 1.5 Gol Alt Olur mu? :: Evet"]
    )

    bets["Her İki Yarıda da 1.5 Gol Alt Olur mu? :: Hayır"] = (
        (~((first_half_total_goal < 1.5) & (second_half_total_goal < 1.5))).sum(),
        test_row["Her İki Yarıda da 1.5 Gol Alt Olur mu? :: Hayır"]
    )

    # ÜST
    bets["Her İki Yarıda da 1.5 Gol Üst Olur mu? :: Evet"] = (
        ((first_half_total_goal >= 1.5) & (second_half_total_goal >= 1.5)).sum(),
        test_row["Her İki Yarıda da 1.5 Gol Üst Olur mu? :: Evet"]
    )

    bets["Her İki Yarıda da 1.5 Gol Üst Olur mu? :: Hayır"] = (
        (~((first_half_total_goal >= 1.5) & (second_half_total_goal >= 1.5))).sum(),
        test_row["Her İki Yarıda da 1.5 Gol Üst Olur mu? :: Hayır"]
    )

    # Karşılıklı Gol (KG Var / KG Yok)
    bets["Karşılıklı Gol :: KG Var"] = (
        ((home_goal > 0) & (away_goal > 0)).sum(),
        test_row["Karşılıklı Gol :: KG Var"]
    )

    bets["Karşılıklı Gol :: KG Yok"] = (
        (~((home_goal > 0) & (away_goal > 0))).sum(),
        test_row["Karşılıklı Gol :: KG Yok"]
    )

    # Karşılıklı Gol ve 2,5 Gol Alt/Üst Kombinasyonu
    total_goal = home_goal + away_goal
    kg_var = (home_goal > 0) & (away_goal > 0)
    kg_yok = ~kg_var

    # 2,5 Alt ve KG Var
    bets["Karşılıklı Gol ve 2,5 Gol Alt/Üst :: 2,5 Alt ve KG Var"] = (
        ((total_goal < 2.5) & kg_var).sum(),
        test_row["Karşılıklı Gol ve 2,5 Gol Alt/Üst :: 2,5 Alt ve KG Var"]
    )

    # 2,5 Alt ve KG Yok
    bets["Karşılıklı Gol ve 2,5 Gol Alt/Üst :: 2,5 Alt ve KG Yok"] = (
        ((total_goal < 2.5) & kg_yok).sum(),
        test_row["Karşılıklı Gol ve 2,5 Gol Alt/Üst :: 2,5 Alt ve KG Yok"]
    )

    # 2,5 Üst ve KG Var
    bets["Karşılıklı Gol ve 2,5 Gol Alt/Üst :: 2,5 Üst ve KG Var"] = (
        ((total_goal >= 2.5) & kg_var).sum(),
        test_row["Karşılıklı Gol ve 2,5 Gol Alt/Üst :: 2,5 Üst ve KG Var"]
    )

    # 2,5 Üst ve KG Yok
    bets["Karşılıklı Gol ve 2,5 Gol Alt/Üst :: 2,5 Üst ve KG Yok"] = (
        ((total_goal >= 2.5) & kg_yok).sum(),
        test_row["Karşılıklı Gol ve 2,5 Gol Alt/Üst :: 2,5 Üst ve KG Yok"]
    )

    # Maç Skoru :: Skor Tahminleri
    possible_scores = [
        (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6),
        (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6),
        (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6),
        (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5),
        (4, 0), (4, 1), (4, 2), (4, 3), (4, 4),
        (5, 0), (5, 1), (5, 2), (5, 3), (5, 4),
        (6, 0), (6, 1), (6, 2)
    ]

    for hg, ag in possible_scores:
        for sep in ["-", ":"]:
            label = f"Maç Skoru :: {hg}{sep}{ag}"
            bets[label] = (
                ((home_goal == hg) & (away_goal == ag)).sum(),
                test_row.get(label, 0)  # Eğer test_row’da bu skor yoksa 0 döner
            )

    # Diğer tüm skorlar (düzeltilmiş versiyon)
    mask = False
    for hg, ag in possible_scores:
        mask |= (home_goal == hg) & (away_goal == ag)

    bets["Maç Skoru :: diğer"] = (
        (~mask).sum(),
        test_row["Maç Skoru :: diğer"]
    )

    # Maç Sonucu :: MS 1 / X / 2
    bets["Maç Sonucu :: MS 1"] = (
        (home_goal > away_goal).sum(),
        test_row["Maç Sonucu :: MS 1"]
    )
    print(bets["Maç Sonucu :: MS 1"])

    bets["Maç Sonucu :: MS 2"] = (
        (away_goal > home_goal).sum(),
        test_row["Maç Sonucu :: MS 2"]
    )
    print(bets["Maç Sonucu :: MS 2"])

    bets["Maç Sonucu :: MS X"] = (
        (home_goal == away_goal).sum(),
        test_row["Maç Sonucu :: MS X"]
    )
    print(bets["Maç Sonucu :: MS X"])

    # Maç Sonucu ve X,X Gol Alt/Üst
    total_goal = home_goal + away_goal

    ms1 = home_goal > away_goal
    ms2 = away_goal > home_goal
    msx = home_goal == away_goal

    threshold_map = {
        "1,5": 1.5,
        "2,5": 2.5,
        "3,5": 3.5,
        "4,5": 4.5
    }

    for label, value in threshold_map.items():
        # MS 1
        bets[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS 1 ve {label} Alt"] = (
            ((ms1) & (total_goal < value)).sum(),
            test_row[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS 1 ve {label} Alt"]
        )
        bets[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS 1 ve {label} Üst"] = (
            ((ms1) & (total_goal >= value)).sum(),
            test_row[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS 1 ve {label} Üst"]
        )

        # MS 2
        bets[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS 2 ve {label} Alt"] = (
            ((ms2) & (total_goal < value)).sum(),
            test_row[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS 2 ve {label} Alt"]
        )
        bets[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS 2 ve {label} Üst"] = (
            ((ms2) & (total_goal >= value)).sum(),
            test_row[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS 2 ve {label} Üst"]
        )

        # MS X
        bets[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS X ve {label} Alt"] = (
            ((msx) & (total_goal < value)).sum(),
            test_row[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS X ve {label} Alt"]
        )
        bets[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS X ve {label} Üst"] = (
            ((msx) & (total_goal >= value)).sum(),
            test_row[f"Maç Sonucu ve {label} Gol Alt/Üst :: MS X ve {label} Üst"]
        )


    # Maç Sonucu ve Karşılıklı Gol Kombinasyonu
    kg_var = (home_goal > 0) & (away_goal > 0)
    kg_yok = ~kg_var

    ms1 = home_goal > away_goal
    ms2 = away_goal > home_goal
    msx = home_goal == away_goal

    # MS 1
    bets["Maç Sonucu ve Karşılıklı Gol :: MS 1 ve Var"] = (
        (ms1 & kg_var).sum(),
        test_row["Maç Sonucu ve Karşılıklı Gol :: MS 1 ve Var"]
    )
    bets["Maç Sonucu ve Karşılıklı Gol :: MS 1 ve Yok"] = (
        (ms1 & kg_yok).sum(),
        test_row["Maç Sonucu ve Karşılıklı Gol :: MS 1 ve Yok"]
    )

    # MS 2
    bets["Maç Sonucu ve Karşılıklı Gol :: MS 2 ve Var"] = (
        (ms2 & kg_var).sum(),
        test_row["Maç Sonucu ve Karşılıklı Gol :: MS 2 ve Var"]
    )
    bets["Maç Sonucu ve Karşılıklı Gol :: MS 2 ve Yok"] = (
        (ms2 & kg_yok).sum(),
        test_row["Maç Sonucu ve Karşılıklı Gol :: MS 2 ve Yok"]
    )

    # MS X
    bets["Maç Sonucu ve Karşılıklı Gol :: MS X ve Var"] = (
        (msx & kg_var).sum(),
        test_row["Maç Sonucu ve Karşılıklı Gol :: MS X ve Var"]
    )
    bets["Maç Sonucu ve Karşılıklı Gol :: MS X ve Yok"] = (
        (msx & kg_yok).sum(),
        test_row["Maç Sonucu ve Karşılıklı Gol :: MS X ve Yok"]
    )


    # Toplam Gol Alt/Üst
    total_goal = home_goal + away_goal

    threshold_map = {
        "0,5": 0.5,
        "1,5": 1.5,
        "2,5": 2.5,
        "3,5": 3.5,
        "4,5": 4.5,
        "5,5": 5.5,
        "6,5": 6.5
    }

    for label, value in threshold_map.items():
        bets[f"Toplam Gol Alt/Üst :: {label} Alt"] = (
            (total_goal < value).sum(),
            test_row[f"Toplam Gol Alt/Üst :: {label} Alt"]
        )
        bets[f"Toplam Gol Alt/Üst :: {label} Üst"] = (
            (total_goal >= value).sum(),
            test_row[f"Toplam Gol Alt/Üst :: {label} Üst"]
        )

    # Toplam Gol Aralığı
    total_goal = home_goal + away_goal

    bets["Toplam Gol Aralığı :: 0-1 Gol"] = (
        (total_goal <= 1).sum(),
        test_row["Toplam Gol Aralığı :: 0-1 Gol"]
    )

    bets["Toplam Gol Aralığı :: 2-3 Gol"] = (
        ((total_goal >= 2) & (total_goal <= 3)).sum(),
        test_row["Toplam Gol Aralığı :: 2-3 Gol"]
    )

    bets["Toplam Gol Aralığı :: 4-5 Gol"] = (
        ((total_goal >= 4) & (total_goal <= 5)).sum(),
        test_row["Toplam Gol Aralığı :: 4-5 Gol"]
    )

    bets["Toplam Gol Aralığı :: 6+ Gol"] = (
        (total_goal >= 6).sum(),
        test_row["Toplam Gol Aralığı :: 6+ Gol"]
    )

    # Toplam Gol Tek / Çift
    total_goal = home_goal + away_goal

    bets["Toplam Gol Tek/Çift :: Tek"] = (
        (total_goal % 2 == 1).sum(),
        test_row["Toplam Gol Tek/Çift :: Tek"]
    )

    bets["Toplam Gol Tek/Çift :: Çift"] = (
        (total_goal % 2 == 0).sum(),
        test_row["Toplam Gol Tek/Çift :: Çift"]
    )


    # Çifte Şans :: ÇŞ 1-2 / 1-X / X-2
    bets["Çifte Şans :: ÇŞ 1-2"] = (
        (home_goal != away_goal).sum(),
        test_row["Çifte Şans :: ÇŞ 1-2"]
    )

    bets["Çifte Şans :: ÇŞ 1-X"] = (
        (home_goal >= away_goal).sum(),
        test_row["Çifte Şans :: ÇŞ 1-X"]
    )

    bets["Çifte Şans :: ÇŞ X-2"] = (
        (away_goal >= home_goal).sum(),
        test_row["Çifte Şans :: ÇŞ X-2"]
    )


    # İkinci Yarı Karşılıklı Gol
    bets["İkinci Yarı Karşılıklı Gol :: 2.Y KG Var"] = (
        ((second_half_home_goal > 0) & (second_half_away_goal > 0)).sum(),
        test_row["İkinci Yarı Karşılıklı Gol :: 2.Y KG Var"]
    )

    bets["İkinci Yarı Karşılıklı Gol :: 2.Y KG Yok"] = (
        (~((second_half_home_goal > 0) & (second_half_away_goal > 0))).sum(),
        test_row["İkinci Yarı Karşılıklı Gol :: 2.Y KG Yok"]
    )

    # İkinci Yarı Sonucu
    bets["İkinci Yarı Sonucu :: 2.Y 1"] = (
        (second_half_home_goal > second_half_away_goal).sum(),
        test_row["İkinci Yarı Sonucu :: 2.Y 1"]
    )

    bets["İkinci Yarı Sonucu :: 2.Y 2"] = (
        (second_half_away_goal > second_half_home_goal).sum(),
        test_row["İkinci Yarı Sonucu :: 2.Y 2"]
    )

    bets["İkinci Yarı Sonucu :: 2.Y X"] = (
        (second_half_home_goal == second_half_away_goal).sum(),
        test_row["İkinci Yarı Sonucu :: 2.Y X"]
    )

    # İlk Yarı / Maç Sonucu
    iy_1 = first_half_home_goal > first_half_away_goal
    iy_2 = first_half_away_goal > first_half_home_goal
    iy_x = first_half_home_goal == first_half_away_goal

    ms_1 = home_goal > away_goal
    ms_2 = away_goal > home_goal
    ms_x = home_goal == away_goal

    bets["İlk Yarı / Maç Sonucu :: 1/1"] = ((iy_1 & ms_1).sum(), test_row["İlk Yarı / Maç Sonucu :: 1/1"])
    bets["İlk Yarı / Maç Sonucu :: 1/2"] = ((iy_1 & ms_2).sum(), test_row["İlk Yarı / Maç Sonucu :: 1/2"])
    bets["İlk Yarı / Maç Sonucu :: 1/X"] = ((iy_1 & ms_x).sum(), test_row["İlk Yarı / Maç Sonucu :: 1/X"])

    bets["İlk Yarı / Maç Sonucu :: 2/1"] = ((iy_2 & ms_1).sum(), test_row["İlk Yarı / Maç Sonucu :: 2/1"])
    bets["İlk Yarı / Maç Sonucu :: 2/2"] = ((iy_2 & ms_2).sum(), test_row["İlk Yarı / Maç Sonucu :: 2/2"])
    bets["İlk Yarı / Maç Sonucu :: 2/X"] = ((iy_2 & ms_x).sum(), test_row["İlk Yarı / Maç Sonucu :: 2/X"])

    bets["İlk Yarı / Maç Sonucu :: X/1"] = ((iy_x & ms_1).sum(), test_row["İlk Yarı / Maç Sonucu :: X/1"])
    bets["İlk Yarı / Maç Sonucu :: X/2"] = ((iy_x & ms_2).sum(), test_row["İlk Yarı / Maç Sonucu :: X/2"])
    bets["İlk Yarı / Maç Sonucu :: X/X"] = ((iy_x & ms_x).sum(), test_row["İlk Yarı / Maç Sonucu :: X/X"])

    # İlk Yarı / Maç Sonucu Skorları
    score_labels = [
        "0-0 0-0", "0-0 0-1", "0-0 0-2", "0-0 0-3", "0-0 1-0", "0-0 1-1", "0-0 1-2",
        "0-0 2-0", "0-0 2-1", "0-0 3-0", "0-1 0-1", "0-1 0-2", "0-1 0-3", "0-1 1-1",
        "0-1 1-2", "0-1 2-1", "0-1 4+", "0-2 0-2", "0-2 0-3", "0-2 1-2", "0-2 4+",
        "0-3 0-3", "0-3 4+", "0:0 4+", "1-0 1-0", "1-0 1-1", "1-0 1-2", "1-0 2-0",
        "1-0 2-1", "1-0 3-0", "1-0 4+", "1-1 1-1", "1-1 1-2", "1-1 2-1", "1-1 4+",
        "1-2 1-2", "1-2 4+", "2-0 2-0", "2-0 2-1", "2-0 3-0", "2-0 4+",
        "2-1 2-1", "2-1 4+", "3-0 3-0", "3-0 4+", "4+ 4+"
    ]

    for label in score_labels:
        try:
            iy_part, ms_part = label.split()

            # İlk Yarı skor
            if iy_part == "4+":
                iy_mask = (first_half_home_goal + first_half_away_goal >= 4)
            elif ":" in iy_part:
                iy_home, iy_away = map(int, iy_part.split(":"))
                iy_mask = (first_half_home_goal == iy_home) & (first_half_away_goal == iy_away)
            else:
                iy_home, iy_away = map(int, iy_part.split("-"))
                iy_mask = (first_half_home_goal == iy_home) & (first_half_away_goal == iy_away)

            # Maç skor
            if ms_part == "4+":
                ms_mask = (home_goal + away_goal >= 4)
            else:
                ms_home, ms_away = map(int, ms_part.replace(":", "-").split("-"))
                ms_mask = (home_goal == ms_home) & (away_goal == ms_away)

            bets[f"İlk Yarı / Maç Sonucu Skorları :: {label}"] = (
                (iy_mask & ms_mask).sum(),
                test_row.get(f"İlk Yarı / Maç Sonucu Skorları :: {label}", 0)
            )

        except Exception as e:
            print(f"Etiket hatası: {label} --> {e}")


    # İlk Yarı Gol Alt/Üst
    first_half_total_goal = first_half_home_goal + first_half_away_goal

    threshold_map = {
        "0,5": 0.5,
        "1,5": 1.5,
        "2,5": 2.5,
        "4,5": 4.5
    }

    for label, value in threshold_map.items():
        bets[f"İlk Yarı Gol Alt/Üst :: İY {label} Alt"] = (
            (first_half_total_goal < value).sum(),
            test_row[f"İlk Yarı Gol Alt/Üst :: İY {label} Alt"]
        )
        bets[f"İlk Yarı Gol Alt/Üst :: İY {label} Üst"] = (
            (first_half_total_goal >= value).sum(),
            test_row[f"İlk Yarı Gol Alt/Üst :: İY {label} Üst"]
        )

    # İlk Yarı Gol Tek/Çift
    first_half_total_goal = first_half_home_goal + first_half_away_goal

    bets["İlk Yarı Gol Tek/Çift :: İY Tek"] = (
        (first_half_total_goal % 2 == 1).sum(),
        test_row["İlk Yarı Gol Tek/Çift :: İY Tek"]
    )

    bets["İlk Yarı Gol Tek/Çift :: İY Çift"] = (
        (first_half_total_goal % 2 == 0).sum(),
        test_row["İlk Yarı Gol Tek/Çift :: İY Çift"]
    )

    # İlk Yarı Karşılıklı Gol Var/Yok
    iy_kg_var = (first_half_home_goal > 0) & (first_half_away_goal > 0)
    iy_kg_yok = ~iy_kg_var

    bets["İlk Yarı Karşılıklı Gol :: İY KG Var"] = (
        iy_kg_var.sum(),
        test_row["İlk Yarı Karşılıklı Gol :: İY KG Var"]
    )

    bets["İlk Yarı Karşılıklı Gol :: İY KG Yok"] = (
        iy_kg_yok.sum(),
        test_row["İlk Yarı Karşılıklı Gol :: İY KG Yok"]
    )


    # İlk Yarı Skoru
    iy_score_list = [
        (0, 0), (0, 1), (0, 2),
        (1, 0), (1, 1), (1, 2),
        (2, 0), (2, 1), (2, 2)
    ]

    # Skor etiketlerini tek tek işle
    mask = False
    for hg, ag in iy_score_list:
        label = f"İlk Yarı Skoru :: {hg}-{ag}"
        bets[label] = (
            ((first_half_home_goal == hg) & (first_half_away_goal == ag)).sum(),
            test_row.get(label, 0)
        )
        mask |= (first_half_home_goal == hg) & (first_half_away_goal == ag)

    # Diğer skorlar (hiçbirine uymayanlar)
    bets["İlk Yarı Skoru :: diğer"] = (
        (~mask).sum(),
        test_row["İlk Yarı Skoru :: diğer"]
    )

    # İlk Yarı Sonucu :: İY 1 / İY 2 / İY X
    bets["İlk Yarı Sonucu :: İY 1"] = (
        (first_half_home_goal > first_half_away_goal).sum(),
        test_row["İlk Yarı Sonucu :: İY 1"]
    )

    bets["İlk Yarı Sonucu :: İY 2"] = (
        (first_half_away_goal > first_half_home_goal).sum(),
        test_row["İlk Yarı Sonucu :: İY 2"]
    )

    bets["İlk Yarı Sonucu :: İY X"] = (
        (first_half_home_goal == first_half_away_goal).sum(),
        test_row["İlk Yarı Sonucu :: İY X"]
    )

    # İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst
    first_half_total_goal = first_half_home_goal + first_half_away_goal

    iy1 = first_half_home_goal > first_half_away_goal
    iy2 = first_half_away_goal > first_half_home_goal
    iyx = first_half_home_goal == first_half_away_goal

    # İY 1
    bets["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY 1 ve 1,5 Alt"] = (
        (iy1 & (first_half_total_goal < 1.5)).sum(),
        test_row["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY 1 ve 1,5 Alt"]
    )
    bets["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY 1 ve 1,5 Üst"] = (
        (iy1 & (first_half_total_goal >= 1.5)).sum(),
        test_row["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY 1 ve 1,5 Üst"]
    )

    # İY 2
    bets["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY 2 ve 1,5 Alt"] = (
        (iy2 & (first_half_total_goal < 1.5)).sum(),
        test_row["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY 2 ve 1,5 Alt"]
    )
    bets["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY 2 ve 1,5 Üst"] = (
        (iy2 & (first_half_total_goal >= 1.5)).sum(),
        test_row["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY 2 ve 1,5 Üst"]
    )

    # İY X
    bets["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY X ve 1,5 Alt"] = (
        (iyx & (first_half_total_goal < 1.5)).sum(),
        test_row["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY X ve 1,5 Alt"]
    )
    bets["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY X ve 1,5 Üst"] = (
        (iyx & (first_half_total_goal >= 1.5)).sum(),
        test_row["İlk Yarı Sonucu ve İY 1,5 Gol Alt/Üst :: İY X ve 1,5 Üst"]
    )


    # İlk Yarı Sonucu ve İlk Yarı Karşılıklı Gol
    iy1 = first_half_home_goal > first_half_away_goal
    iy2 = first_half_away_goal > first_half_home_goal
    iyx = first_half_home_goal == first_half_away_goal

    iy_kg_var = (first_half_home_goal > 0) & (first_half_away_goal > 0)
    iy_kg_yok = ~iy_kg_var

    # İY 1
    bets["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 1 ve Var"] = (
        (iy1 & iy_kg_var).sum(),
        test_row["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 1 ve Var"]
    )
    bets["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 1 ve Yok"] = (
        (iy1 & iy_kg_yok).sum(),
        test_row["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 1 ve Yok"]
    )

    # İY 2
    bets["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 2 ve Var"] = (
        (iy2 & iy_kg_var).sum(),
        test_row["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 2 ve Var"]
    )
    bets["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 2 ve Yok"] = (
        (iy2 & iy_kg_yok).sum(),
        test_row["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 2 ve Yok"]
    )

    # İY X
    bets["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY X ve Var"] = (
        (iyx & iy_kg_var).sum(),
        test_row["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY X ve Var"]
    )
    bets["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY X ve Yok"] = (
        (iyx & iy_kg_yok).sum(),
        test_row["İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY X ve Yok"]
    )

    # İlk Yarı Çifte Şans
    iy1 = first_half_home_goal > first_half_away_goal
    iy2 = first_half_away_goal > first_half_home_goal
    iyx = first_half_home_goal == first_half_away_goal

    bets["İlk Yarı Çifte Şans :: İY 1-2"] = (
        (iy1 | iy2).sum(),
        test_row["İlk Yarı Çifte Şans :: İY 1-2"]
    )

    bets["İlk Yarı Çifte Şans :: İY 1-X"] = (
        (iy1 | iyx).sum(),
        test_row["İlk Yarı Çifte Şans :: İY 1-X"]
    )

    bets["İlk Yarı Çifte Şans :: İY X-2"] = (
        (iyx | iy2).sum(),
        test_row["İlk Yarı Çifte Şans :: İY X-2"]
    )


    # Sonuçları saklayacağımız boş liste → sonra df yapacağız
    value_bets_rows = []

    for bet_name, (prob, odds) in bets.items():
        ev = (prob * odds) / 100  # yüzde olduğu için 100'e bölüyoruz
        print(f"{bet_name}: Olasılık %{prob:.2f}, Oran {odds}, EV: {ev:.2f}")
        if ev > 1:
            value_bets.loc[len(value_bets)] = {
                "match_date": test_row["match_date"],
                "match_time": test_row["match_time"],
                "match_id": test_row["match_id"],
                "hometeam": test_row["homeTeam"],
                "awayteam": test_row["awayTeam"],
                "bet_name": bet_name,
                "probability": prob,
                "odds": odds
            }



    print(flag, bets)
    flag += 1
    if flag == 50:
        break


    # # -- 5. Sonuçları yazdır
    # print(f"Test Maçı: {test_row['homeTeam']} vs {test_row['awayTeam']}")
    # print(f"\nTop 100 en yakın maç bazında MS olasılıkları:")
    # print(f"  MS 1: %{ms1_prob * 100:.2f}")
    # print(f"  MS X: %{msx_prob * 100:.2f}")
    # print(f"  MS 2: %{ms2_prob * 100:.2f}")
    # print(test_row["Maç Sonucu :: MS 1"])
    # print(test_row["Maç Sonucu :: MS X"])
    # print(test_row["Maç Sonucu :: MS 2"])
    # print(f"  MS 1: %{test_row["Maç Sonucu :: MS 1"]*ms1_prob * 100:.2f}")
    # print(f"  MS X: %{test_row["Maç Sonucu :: MS X"]*msx_prob * 100:.2f}")
    # print(f"  MS 2: %{test_row["Maç Sonucu :: MS 2"]*ms2_prob * 100:.2f}")

    # values = [test_row["Maç Sonucu :: MS 1"]*ms1_prob * 100,
    #         test_row["Maç Sonucu :: MS X"]*msx_prob * 100,
    #         test_row["Maç Sonucu :: MS 2"]*ms2_prob * 100
    #         ]
    # if max(values) >= 0:
    #     print(f"Test Maçı: {test_row['homeTeam']} vs {test_row['awayTeam']}")
    #     print(f"\nTop 100 en yakın maç bazında MS olasılıkları:")
    #     print(f"  MS 1: %{ms1_prob * 100:.2f}")
    #     print(f"  MS X: %{msx_prob * 100:.2f}")
    #     print(f"  MS 2: %{ms2_prob * 100:.2f}")
    #     print(test_row["Maç Sonucu :: MS 1"])
    #     print(test_row["Maç Sonucu :: MS X"])
    #     print(test_row["Maç Sonucu :: MS 2"])
    #     print(f"  MS 1: %{test_row["Maç Sonucu :: MS 1"]*ms1_prob * 100:.2f}")
    #     print(f"  MS X: %{test_row["Maç Sonucu :: MS X"]*msx_prob * 100:.2f}")
    #     print(f"  MS 2: %{test_row["Maç Sonucu :: MS 2"]*ms2_prob * 100:.2f}")
    #     print(values)
    #     print("-"*50)

53 36 11
(53, 1.61)
(11, 3.68)
(36, 2.96)
Deplasman Gol Alt/Üst :: Dep 0,5 Alt: Olasılık %49.00, Oran 1.98, EV: 0.97
Deplasman Gol Alt/Üst :: Dep 0,5 Üst: Olasılık %51.00, Oran 1.38, EV: 0.70
Deplasman Gol Alt/Üst :: Dep 1,5 Alt: Olasılık %78.00, Oran nan, EV: nan
Deplasman Gol Alt/Üst :: Dep 1,5 Üst: Olasılık %22.00, Oran nan, EV: nan
Deplasman Gol Alt/Üst :: Dep 2,5 Alt: Olasılık %96.00, Oran nan, EV: nan
Deplasman Gol Alt/Üst :: Dep 2,5 Üst: Olasılık %4.00, Oran nan, EV: nan
Deplasman Gol Alt/Üst :: Dep 3,5 Alt: Olasılık %98.00, Oran nan, EV: nan
Deplasman Gol Alt/Üst :: Dep 3,5 Üst: Olasılık %2.00, Oran nan, EV: nan
Deplasman Gol Alt/Üst :: Dep 4,5 Alt: Olasılık %99.00, Oran nan, EV: nan
Deplasman Gol Alt/Üst :: Dep 4,5 Üst: Olasılık %1.00, Oran nan, EV: nan
Deplasman Gol Alt/Üst :: Dep 6,5 Alt: Olasılık %100.00, Oran nan, EV: nan
Deplasman Gol Alt/Üst :: Dep 6,5 Üst: Olasılık %0.00, Oran nan, EV: nan
Deplasman Gol Yemeden Kazanır mı? :: Evet: Olasılık %6.00, Oran nan, EV: nan
Depl

In [81]:
bets

{'Deplasman Gol Alt/Üst :: Dep 0,5 Alt': (37, nan),
 'Deplasman Gol Alt/Üst :: Dep 0,5 Üst': (63, nan),
 'Deplasman Gol Alt/Üst :: Dep 1,5 Alt': (70, 1.43),
 'Deplasman Gol Alt/Üst :: Dep 1,5 Üst': (30, 1.94),
 'Deplasman Gol Alt/Üst :: Dep 2,5 Alt': (87, nan),
 'Deplasman Gol Alt/Üst :: Dep 2,5 Üst': (13, nan),
 'Deplasman Gol Alt/Üst :: Dep 3,5 Alt': (93, nan),
 'Deplasman Gol Alt/Üst :: Dep 3,5 Üst': (7, nan),
 'Deplasman Gol Alt/Üst :: Dep 4,5 Alt': (98, nan),
 'Deplasman Gol Alt/Üst :: Dep 4,5 Üst': (2, nan),
 'Deplasman Gol Alt/Üst :: Dep 6,5 Alt': (100, nan),
 'Deplasman Gol Alt/Üst :: Dep 6,5 Üst': (0, nan),
 'Deplasman Gol Yemeden Kazanır mı? :: Evet': (9, nan),
 'Deplasman Gol Yemeden Kazanır mı? :: Hayır': (91, nan),
 'Deplasman Hangi Yarıda Daha Fazla Gol Atar? :: 1. Yarı': (16, nan),
 'Deplasman Hangi Yarıda Daha Fazla Gol Atar? :: 2. Yarı': (43, nan),
 'Deplasman Hangi Yarıda Daha Fazla Gol Atar? :: Eşit': (41, nan),
 'Deplasman Her İki Yarıyı da Kazanır mı? :: Evet': (6,

In [82]:
value_bets.to_csv("value_bets.csv", index=False)

In [83]:
# value_bets["probability"] *= 2

In [84]:
sorted_value_bets = value_bets.sort_values(by=["probability"], ascending=False)

In [85]:
sorted_value_bets["EV"] = (sorted_value_bets["probability"] * sorted_value_bets["odds"]) / 100

In [86]:
for idx, row in sorted_value_bets.iterrows():
    if idx == 35:
        print(row)

match_date                        2025-07-05
match_time                          07:00:00
match_id                             2186262
hometeam                      Weston Workers
awayteam                Belmont Swansea Unit
bet_name       Toplam Gol Alt/Üst :: 3,5 Alt
probability                               55
odds                                    1.93
EV                                    1.0615
Name: 35, dtype: object


In [87]:
sorted_value_bets.head()
sorted_value_bets = sorted_value_bets.copy()

In [88]:
# Kelly fraksiyonunu hesapla: f* = (b * p − q) / b
p = sorted_value_bets["probability"] / 100
q = 1 - p
b = sorted_value_bets["odds"] - 1

sorted_value_bets["Kelly"] = ((b * p - q) / b).clip(lower=0)
sorted_value_bets["stake"] = sorted_value_bets["Kelly"] * 100
sorted_value_bets["bankroll_if_win"] = 1 + (sorted_value_bets["Kelly"] * (sorted_value_bets["odds"] - 1))
sorted_value_bets["bankroll_if_lose"] = 1 - sorted_value_bets["Kelly"]
p = sorted_value_bets["probability"] / 100
q = 1 - p

sorted_value_bets["expected_bankroll"] = (
    p * sorted_value_bets["bankroll_if_win"] +
    q * sorted_value_bets["bankroll_if_lose"]
)


In [89]:
sorted_value_bets.sort_values(by=["EV"], ascending=False).head(5)

,match_date,match_time,match_id,hometeam,awayteam,bet_name,probability,odds,EV,Kelly,stake,bankroll_if_win,bankroll_if_lose,expected_bankroll
115,2025-08-26,22:00:00,2305677,Sheffield Wed.,Leeds Utd.,Maç Skoru :: 2:0,4,99.00,3.9600,0.030204,3.020408,3.9600,0.969796,1.089404
39,2025-08-28,20:00:00,2284511,Samsunspor,Panathinaikos,Maç Skoru :: 0:5,2,130.00,2.6000,0.012403,1.240310,2.6000,0.987597,1.019845
118,2025-08-26,22:00:00,2305677,Sheffield Wed.,Leeds Utd.,Maç Skoru :: 3:1,2,130.00,2.6000,0.012403,1.240310,2.6000,0.987597,1.019845
200,2025-08-22,12:30:00,2242924,Sydney FC NPL Gençle,Blacktown Gençler,İkinci Yarı Sonucu :: 2.Y 2,26,9.98,2.5948,0.177595,17.759465,2.5948,0.822405,1.283228
112,2025-08-26,22:00:00,2305677,Sheffield Wed.,Leeds Utd.,Hangi Takım Kaç Farkla Kazanır :: Ev 2 Fark,7,34.50,2.4150,0.042239,4.223881,2.4150,0.957761,1.059768


In [90]:
sorted_value_bets

,match_date,match_time,match_id,hometeam,awayteam,bet_name,probability,odds,EV,Kelly,stake,bankroll_if_win,bankroll_if_lose,expected_bankroll
198,2025-08-22,12:30:00,2242924,Sydney FC NPL Gençle,Blacktown Gençler,"Toplam Gol Alt/Üst :: 5,5 Alt",98,1.07,1.0486,0.694286,69.428571,1.0486,0.305714,1.033742
156,2025-08-22,13:15:00,2317915,FC Strathmore,Uni Hill Eagles FC,"Toplam Gol Alt/Üst :: 5,5 Alt",92,1.29,1.1868,0.644138,64.413793,1.1868,0.355862,1.120325
194,2025-08-22,12:30:00,2242924,Sydney FC NPL Gençle,Blacktown Gençler,"Deplasman Gol Alt/Üst :: Dep 2,5 Alt",87,1.36,1.1832,0.508889,50.888889,1.1832,0.491111,1.093228
197,2025-08-22,12:30:00,2242924,Sydney FC NPL Gençle,Blacktown Gençler,"Toplam Gol Alt/Üst :: 4,5 Alt",87,2.02,1.7574,0.742549,74.254902,1.7574,0.257451,1.562407
195,2025-08-22,12:30:00,2242924,Sydney FC NPL Gençle,Blacktown Gençler,"Ev Sahibi Gol Alt/Üst :: Ev 2,5 Alt",85,1.22,1.0370,0.168182,16.818182,1.0370,0.831818,1.006223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2025-08-28,21:45:00,2284557,Linfield,Shelbourne,Maç Skoru :: 0:5,1,130.00,1.3000,0.002326,0.232558,1.3000,0.997674,1.000698
119,2025-08-26,22:00:00,2305677,Sheffield Wed.,Leeds Utd.,Maç Skoru :: 3:3,1,130.00,1.3000,0.002326,0.232558,1.3000,0.997674,1.000698
15,2025-08-28,20:00:00,2284524,Rapid Wien,ETO Gyor,Maç Skoru :: 1:4,1,130.00,1.3000,0.002326,0.232558,1.3000,0.997674,1.000698
71,2025-08-27,22:00:00,2284584,Club Brugge,Rangers,Maç Skoru :: 2:4,1,130.00,1.3000,0.002326,0.232558,1.3000,0.997674,1.000698


In [91]:
value_bets_top_prob = (
    sorted_value_bets
    .sort_values("probability", ascending=False)
    .groupby("match_id", as_index=False)
    .first()
    )

value_bets_top_bankroll = (
    sorted_value_bets
    .sort_values("expected_bankroll", ascending=False)
    .groupby("match_id", as_index=False)
    .first()
    )

In [92]:
# value_bets = value_bets_top.sort_values(by=["expected_bankroll"], ascending=False)

In [93]:
sorted_vals_prob = value_bets_top_prob.sort_values(by=["probability"], ascending=False)
sorted_vals_prob

,match_id,match_date,match_time,hometeam,awayteam,bet_name,probability,odds,EV,Kelly,stake,bankroll_if_win,bankroll_if_lose,expected_bankroll
13,2242924,2025-08-22,12:30:00,Sydney FC NPL Gençle,Blacktown Gençler,"Toplam Gol Alt/Üst :: 5,5 Alt",98,1.07,1.0486,0.694286,69.428571,1.0486,0.305714,1.033742
28,2317915,2025-08-22,13:15:00,FC Strathmore,Uni Hill Eagles FC,"Toplam Gol Alt/Üst :: 5,5 Alt",92,1.29,1.1868,0.644138,64.413793,1.1868,0.355862,1.120325
22,2284579,2025-08-26,19:45:00,K. Almaty,Celtic,"Deplasman Gol Alt/Üst :: Dep 2,5 Alt",82,1.26,1.0332,0.127692,12.769231,1.0332,0.872308,1.004239
24,2305677,2025-08-26,22:00:00,Sheffield Wed.,Leeds Utd.,Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır,75,1.44,1.0800,0.181818,18.181818,1.0800,0.818182,1.014545
2,2146330,2025-05-31,07:30:00,Clarence Zebras,Launceston,"Toplam Gol Alt/Üst :: 4,5 Alt",73,1.64,1.1972,0.308125,30.812500,1.1972,0.691875,1.060762
12,2242719,2025-08-22,12:30:00,Brisbane City (K),Sunshine Coast Wande,"Ev Sahibi Gol Alt/Üst :: Ev 0,5 Üst",71,1.42,1.0082,0.019524,1.952381,1.0082,0.980476,1.000160
8,2202048,2025-07-19,07:00:00,South Hobart,Devonport City,"Deplasman Gol Alt/Üst :: Dep 1,5 Alt",70,1.43,1.0010,0.002326,0.232558,1.0010,0.997674,1.000002
3,2157748,2025-05-18,00:00:00,Dep. Tachira,Carabobo FC,"Deplasman Gol Alt/Üst :: Dep 0,5 Üst",67,1.58,1.0586,0.101034,10.103448,1.0586,0.898966,1.005921
27,2315576,2025-08-27,23:00:00,Carabobo FC,Portuguesa,"Deplasman Gol Alt/Üst :: Dep 0,5 Üst",66,1.61,1.0626,0.102623,10.262295,1.0626,0.897377,1.006424
11,2223375,2025-07-04,01:00:00,Strongest,Cdt Real Oruro,"Ev Sahibi Gol Alt/Üst :: Ev 2,5 Alt",66,1.56,1.0296,0.052857,5.285714,1.0296,0.947143,1.001565


In [94]:
sorted_vals_bankroll = value_bets_top_bankroll.sort_values(by=["expected_bankroll"], ascending=False)
sorted_vals_bankroll

,match_id,match_date,match_time,hometeam,awayteam,bet_name,probability,odds,EV,Kelly,stake,bankroll_if_win,bankroll_if_lose,expected_bankroll
13,2242924,2025-08-22,12:30:00,Sydney FC NPL Gençle,Blacktown Gençler,"Toplam Gol Alt/Üst :: 4,5 Alt",87,2.02,1.7574,0.742549,74.254902,1.7574,0.257451,1.562407
28,2317915,2025-08-22,13:15:00,FC Strathmore,Uni Hill Eagles FC,"Ev Sahibi Gol Alt/Üst :: Ev 1,5 Alt",76,1.98,1.5048,0.515102,51.510204,1.5048,0.484898,1.260024
12,2242719,2025-08-22,12:30:00,Brisbane City (K),Sunshine Coast Wande,Maç Sonucu :: MS 1,40,4.09,1.6360,0.205825,20.582524,1.6360,0.794175,1.130905
24,2305677,2025-08-26,22:00:00,Sheffield Wed.,Leeds Utd.,Maç Skoru :: 2:0,4,99.00,3.9600,0.030204,3.020408,3.9600,0.969796,1.089404
2,2146330,2025-05-31,07:30:00,Clarence Zebras,Launceston,"Toplam Gol Alt/Üst :: 4,5 Alt",73,1.64,1.1972,0.308125,30.812500,1.1972,0.691875,1.060762
3,2157748,2025-05-18,00:00:00,Dep. Tachira,Carabobo FC,Maç Sonucu :: MS 2,32,4.49,1.4368,0.125158,12.515759,1.4368,0.874842,1.054669
22,2284579,2025-08-26,19:45:00,K. Almaty,Celtic,"Deplasman Gol Alt/Üst :: Dep 1,5 Alt",60,2.06,1.2360,0.222642,22.264151,1.2360,0.777358,1.052543
27,2315576,2025-08-27,23:00:00,Carabobo FC,Portuguesa,Maç Sonucu :: MS 2,30,4.60,1.3800,0.105556,10.555556,1.3800,0.894444,1.040111
1,2144790,2025-05-30,13:00:00,Broad. Magic,New Lambton FC,Karşılıklı Gol :: KG Yok,59,1.99,1.1741,0.175859,17.585859,1.1741,0.824141,1.030617
19,2284511,2025-08-28,20:00:00,Samsunspor,Panathinaikos,Maç Skoru :: 0:5,2,130.00,2.6000,0.012403,1.240310,2.6000,0.987597,1.019845


In [95]:
sorted_vals_prob_time = sorted_vals_prob.sort_values(by=["match_date", "match_time"], ascending=True)
sorted_vals_prob_time
sorted_vals_prob_time.to_csv("value_bets_by_prob.csv", index=False)

In [96]:
sorted_vals_bankroll_time = sorted_vals_bankroll.sort_values(by=["match_date", "match_time"], ascending=True)
sorted_vals_bankroll_time
sorted_vals_bankroll_time.to_csv("value_bets_by_bankroll.csv", index=False)